### Magic Formula Implementation 

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
tickers = ["MMM","AXP","AAPL","BA","CAT","CVX","CSCO","KO","DIS","DWDP",
           "XOM","GE","GS","HD","IBM","INTC","JNJ","JPM","MCD","MRK",
           "MSFT","NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT"]

In [18]:
financial_dir = {}

for ticker in tickers:
    try:
        print("Scraping financial data statement data for ", ticker)
        # getting balance sheet data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Balance%20Sheet&sort=desc".format(ticker)
        df1 = pd.read_excel(url)
        # getting income statement data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Income%20Statement&sort=desc".format(ticker)
        df2 = pd.read_excel(url)        
        # getting cashflow statement data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Cash%20Flow&sort=desc".format(ticker)
        df3 = pd.read_excel(url)

        #getting key statistics data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        headers = {'User-Agent':"Chrome/104.0.5112.110"}
        page = requests.get(url, headers=headers)
        page_content = page.content
        soup = BeautifulSoup(page_content, 'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c)"})
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2]) > 0:
                    temp_dir[row.get_text(separator='|').split("|")[0]] = row.get_text(separator='|').split("|")[-1]
        df4 = pd.DataFrame(temp_dir.items(), columns=df3.columns[0:2])
        df4.iloc[:,1] = df4.iloc[:,1].replace({'M': 'E+03', 'B': 'E+06', 'T': 'E+09', '%': 'E-02'}, regex=True)
        df4.iloc[:,1] = pd.to_numeric(df4.iloc[:,1], errors="coerce")
        df4 = df4[df4["Unnamed: 0"].isin(["Market Cap (intraday)", "Forward Annual Dividend Yield"])]

        #combining all extracted information with the corresponding ticker
        df = pd.concat([df1, df2, df3, df4]).iloc[:,[0,1]]
        columns = df.columns.values

        for i in range(len(columns)):
            if columns[i] == "Unnamed: 0":
                columns[i] = "heading"
            else:
                columns[i] = columns[i].strftime("%Y-%m-%d")
        df.columns = columns
        df.set_index("heading", inplace=True)
        financial_dir[ticker] = df
    except Exception as e:
        print(ticker, ":", e)

Scraping financial data statement data for  MMM
Scraping financial data statement data for  AXP
Scraping financial data statement data for  AAPL
Scraping financial data statement data for  BA
Scraping financial data statement data for  CAT
Scraping financial data statement data for  CVX
Scraping financial data statement data for  CSCO
Scraping financial data statement data for  KO
Scraping financial data statement data for  DIS
Scraping financial data statement data for  DWDP
DWDP : HTTP Error 404: Not Found
Scraping financial data statement data for  XOM
Scraping financial data statement data for  GE
Scraping financial data statement data for  GS
Scraping financial data statement data for  HD
Scraping financial data statement data for  IBM
Scraping financial data statement data for  INTC
Scraping financial data statement data for  JNJ
Scraping financial data statement data for  JPM
Scraping financial data statement data for  MCD
Scraping financial data statement data for  MRK
Scraping

In [19]:
# sample
financial_dir['V'].index.values

array(['Cash and Short Term Investments', 'Receivables',
       'Other current assets', 'Total current assets',
       'Property, Plant, Equpment (Net)', 'Long-Term Investments',
       'Goodwill and Intangible Assets (Total)',
       'Long-term assets (Other)', 'Total non-current assets',
       'Total Assets', 'Accounts Payable', 'Dividends Payable',
       'Accrued Expenses', 'Current Part of Debt',
       'Other current liabilities', 'Total current liabilities',
       'Long Term Debt (Total)', 'Long Term Tax Liability (Deferred)',
       'Non-current Liabilities (Other)', 'Total non-current liabilities',
       'Total liabilities', 'Additional Paid In Capital',
       'Preferred Stock (Total)', 'Retained Earnings', 'Treasury Stock',
       'Shareholders Equity (Other)', 'Common Equity (Total)',
       'Shareholders Equity (Total)',
       'Shareholders Equity and Liabilities (Total)', 'Shares (Common)',
       'Shares (Preferred)', 'Shareholders Equity (Tangible)', 'Net Debt',
   

In [20]:
# Creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & Amortization",
         "Net Cash/Marketcap",
         "Net Income Common",
         "Operating Cash Flow",
         "Capital expenditures",
         "Total current assets",
         "Total current liabilities",
         "Property, Plant, Equpment (Net)",
         "Shareholders Equity (Total)",
         "Long Term Debt (Total)",
         "Dividends Paid (Total)"] # change as required

indx = ["EBITDA", "D&A", "MarketCap", "NetIncome", "CashFlowOps", "Capex", "CurrAsset", "CurrLiab", "PPE", "BookValue", "TotDebt", "DivYield"]

def info_filter(df, stats, indx):
    """ function to filter relevant financial information 
    df = dataframe to be filtered
    stats = headings to filter
    indx = rename long headings
    lookback = number of years of data to be retained"""
    for stat in stats:
        if stat not in df.index:
            return
    df_new = df.loc[stats,:]
    df_new.rename(dict(zip(stats, indx)), inplace=True)
    return df_new

In [21]:
# applying filtering to the financials and calculating relevant financial metrics for each stock
transformed_df = {}
for ticker in financial_dir:
    transformed_df[ticker] = info_filter(financial_dir[ticker], stats, indx)
    if transformed_df[ticker] is None:
        del transformed_df[ticker]
        continue
    transformed_df[ticker].loc["EBIT",:] = transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]
    transformed_df[ticker].loc["TEV",:] = transformed_df[ticker].loc["MarketCap",:] + transformed_df[ticker].loc["TotDebt",:] - (transformed_df[ticker].loc["CurrAsset",:] - transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["EarningYield",:] = transformed_df[ticker].loc["EBIT",:]/transformed_df[ticker].loc["TEV",:]
    transformed_df[ticker].loc["FCFYield",:] = (transformed_df[ticker].loc["CashFlowOps",:] - transformed_df[ticker].loc["Capex",:])/transformed_df[ticker].loc["MarketCap",:]
    transformed_df[ticker].loc["ROC",:] = (transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]) / (transformed_df[ticker].loc["PPE",:] + transformed_df[ticker].loc["CurrAsset",:] - transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["BookToMkt",:] = transformed_df[ticker].loc["BookValue",:] / transformed_df[ticker].loc["MarketCap",:]

In [22]:
## Output DataFrame
final_stats_val_df = pd.DataFrame(columns=transformed_df.keys())
for key in transformed_df:
    final_stats_val_df[key] = transformed_df[key].values.flatten()
    final_stats_val_df.set_index(transformed_df[key].index, inplace=True)

In [23]:
# Finding value stocks based on Greenblatt Magic Formula
final_stats_val_df.loc["CombRank", :] = final_stats_val_df.loc["EarningYield", :].rank(ascending=False, na_option='bottom') + final_stats_val_df.loc["ROC", :].rank(ascending=False, na_option='bottom')
final_stats_val_df.loc["MagicFormulaRank", :] = final_stats_val_df.loc["CombRank", :].rank(method='first')
value_stocks = final_stats_val_df.loc["MagicFormulaRank", :].sort_values()

In [24]:
print("Value Stocks based on Greenblatt's Magic Formula")
print(value_stocks)

Value Stocks based on Greenblatt's Magic Formula
AAPL     1.0
V        2.0
JNJ      3.0
PFE      4.0
PG       5.0
HD       6.0
MMM      7.0
INTC     8.0
IBM      9.0
KO      10.0
MRK     11.0
CAT     12.0
CVX     13.0
CSCO    14.0
XOM     15.0
WMT     16.0
VZ      17.0
MCD     18.0
GE      19.0
MSFT    20.0
NKE     21.0
UNH     22.0
DIS     23.0
GS      24.0
BA      25.0
Name: MagicFormulaRank, dtype: float64


In [25]:
# Finding highest dividend yield stocks
high_dividend_stocks = final_stats_val_df.loc["DivYield",:].sort_values(ascending=False)
print("Highest Dividend Paying Stocks")
print(high_dividend_stocks)

Highest Dividend Paying Stocks
GE     -5.750000e+08
NKE    -1.837000e+09
CAT    -2.332000e+09
GS     -2.725000e+09
V      -2.798000e+09
MMM    -3.420000e+09
MCD    -3.919000e+09
UNH    -5.280000e+09
INTC   -5.644000e+09
IBM    -5.869000e+09
WMT    -6.152000e+09
CSCO   -6.224000e+09
MRK    -6.610000e+09
HD     -6.985000e+09
KO     -7.252000e+09
PFE    -8.729000e+09
PG     -8.770000e+09
CVX    -1.021500e+10
VZ     -1.044500e+10
JNJ    -1.103200e+10
AAPL   -1.446700e+10
XOM    -1.514800e+10
MSFT   -1.813500e+10
BA               NaN
DIS              NaN
Name: DivYield, dtype: float64


In [26]:
# Magic formula and Dividend Yield Combined
final_stats_val_df.loc["CombinedRank",:] = final_stats_val_df.loc["EarningYield",:].rank(ascending=False, method='first') + final_stats_val_df.loc["ROC",:].rank(ascending=False, method='first') + final_stats_val_df.loc["DivYield",:].rank(ascending=False, method='first')
value_high_div_stocks = final_stats_val_df.T.sort_values("CombinedRank").loc[:, ["EarningYield", "ROC", "DivYield", "CombinedRank"]]
print("Magic Formula and Dividend Yield Combined")
print(value_high_div_stocks)

Magic Formula and Dividend Yield Combined
heading  EarningYield       ROC      DivYield  CombinedRank
V            1.951665  1.085373 -2.798000e+09          11.0
MMM          0.776837  0.476420 -3.420000e+09          25.0
CAT          0.560805  0.271150 -2.332000e+09          26.0
AAPL         0.990211  1.850294 -1.446700e+10          27.0
INTC         6.163145  0.232568 -5.644000e+09          28.0
PFE          0.958954  0.598480 -8.729000e+09          30.0
IBM          0.143335  4.351301 -5.869000e+09          30.0
HD           0.553913  0.730756 -6.985000e+09          31.0
JNJ          1.607209  0.687379 -1.103200e+10          31.0
PG           0.522611  1.834033 -8.770000e+09          32.0
MRK          0.535481  0.501339 -6.610000e+09          35.0
KO           0.290195  0.823652 -7.252000e+09          36.0
GE           0.231156  0.148514 -5.750000e+08          36.0
NKE         -1.174905  0.269762 -1.837000e+09          40.0
WMT          0.486812  0.267526 -6.152000e+09          40.